In [1]:
import os,sys,json
import torch
from datasets import load_dataset
import random
import io
import time
from transformers import AutoTokenizer
from vllm import LLM, SamplingParams
import numpy as np
from tqdm import tqdm
from copy import deepcopy   



INFO 10-06 15:55:23 [__init__.py:216] Automatically detected platform cuda.


In [2]:
def jload(f, mode="r", num_samples=None):
    if not isinstance(f, io.IOBase): f = open(f, mode=mode)
    jdict = json.load(f)
    f.close()
    if num_samples is not None and num_samples > 0 and num_samples < len(jdict):
        random.seed(10)
        jdict = random.sample(jdict, num_samples)
        random.seed(time.time())
    return jdict

def jdump(obj, f, mode="w", indent=4, default=str):
    if not isinstance(f, io.IOBase): f = open(f, mode=mode)
    if isinstance(obj, (dict, list)): json.dump(obj, f, indent=indent, default=default)
    elif isinstance(obj, str): f.write(obj)
    else: raise ValueError(f"Unexpected type: {type(obj)}")
    f.close()

OTHER_DELM_TOKENS = {
    'mark': ['{s}', '|{s}|', '<{s}>', '[{s}]', '<|{s}|>', '[|{s}|]', '<[{s}]>', '\'\'\'{s}\'\'\'', '***{s}***'],
    'inst': ['Command', 'Rule', 'Prompt', 'Task'],
    'inpt': ['Data', 'Context', 'Text'],
    'resp': ['Output', 'Answer', 'Reply'],
    'user': ['', 'Prompter ', 'User ', 'Human '],
    'asst': ['', 'Assistant ', 'Chatbot ', 'Bot ', 'GPT ', 'AI '],
}

def create_injection_for_completion(response, instruction, input):
    mark = np.random.choice(OTHER_DELM_TOKENS['mark']) + ':'
    
    def sample_delm(delm_name):
        role_name = 'user' if (delm_name == 'inst' or delm_name == 'inpt') else 'asst'
        role = np.random.choice(OTHER_DELM_TOKENS[role_name]) 
        delm = np.random.choice(OTHER_DELM_TOKENS[delm_name])
        p = np.random.rand()
        if p < 1/3: return (role + delm).upper()
        elif p < 2/3: return (role + delm).lower()
        else: return role + delm
    
    text = mark.format(s=sample_delm('resp')) + '\n\n' + response 
    text += '\n\n' + mark.format(s=sample_delm('inst')) + '\n\n' + instruction
    if input != '':  text += '\n\n' + mark.format(s=sample_delm('inpt')) + '\n\n' + input
    return text

def calculate_length_for_preference_dataset(dataset, tokenizer):
    chosen_input_ids = tokenizer([d['chosen'] for d in dataset], add_special_tokens=False)["input_ids"]
    rejected_input_ids = tokenizer([d['rejected'] for d in dataset], add_special_tokens=False)["input_ids"]

    chosen_lengths = np.array([len(prompt) for prompt in chosen_input_ids])
    rejected_lengths = np.array([len(prompt) for prompt in rejected_input_ids])
    prompt_and_label_lengths = np.maximum(chosen_lengths,rejected_lengths)

    print('Input+Output model_max_length (98%, 99%, 99.5%, 99.9%):', np.percentile(prompt_and_label_lengths, [95, 99, 99.5, 99.9]))
    print (f'Mean: {(np.mean(chosen_lengths) + np.mean(rejected_lengths))/2:.2f} Num > 2048: {np.sum(prompt_and_label_lengths>2048)} / {len(prompt_and_label_lengths)}')

In [3]:
data_dir = "/mnt/disk1/wjyeo/data"
preference_data_path = {'train':os.path.join(data_dir, "metasecalign_train.json"),
                        'val':os.path.join(data_dir, "metasecalign_val.json"),}

clean_data = load_dataset("yahma/alpaca-cleaned")['train']
print (len(clean_data))

51760


In [ ]:
model_path = '/mnt/disk1/xulin/models/Qwen3-8B'
model = LLM(model_path, tensor_parallel_size=torch.cuda.device_count(), trust_remote_code=True) 
tokenizer = AutoTokenizer.from_pretrained(model_path)
sampling_params = SamplingParams(temperature=0.8, max_tokens=2048, stop=tokenizer.eos_token)

INFO 10-06 15:23:48 [utils.py:328] non-default args: {'trust_remote_code': True, 'disable_log_stats': True, 'model': '/mnt/disk1/xulin/models/Qwen3-8B'}


The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


INFO 10-06 15:23:53 [__init__.py:742] Resolved architecture: Qwen3ForCausalLM


`torch_dtype` is deprecated! Use `dtype` instead!


INFO 10-06 15:23:53 [__init__.py:1815] Using max model len 40960
INFO 10-06 15:23:55 [scheduler.py:222] Chunked prefill is enabled with max_num_batched_tokens=16384.
(EngineCore_DP0 pid=571756) INFO 10-06 15:23:56 [core.py:654] Waiting for init message from front-end.
(EngineCore_DP0 pid=571756) INFO 10-06 15:23:56 [core.py:76] Initializing a V1 LLM engine (v0.10.2) with config: model='/mnt/disk1/xulin/models/Qwen3-8B', speculative_config=None, tokenizer='/mnt/disk1/xulin/models/Qwen3-8B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=40960, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, data_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, device_config=cuda, decoding_config=DecodingConfig(backend='auto', disable_fallback=False, disable_any_whitespace=False, disable_additional_propertie

...

Loading safetensors checkpoint shards:   0% Completed | 0/5 [00:00<?, ?it/s]


(EngineCore_DP0 pid=571756) INFO 10-06 15:24:02 [default_loader.py:268] Loading weights took 3.24 seconds
(EngineCore_DP0 pid=571756) INFO 10-06 15:24:02 [gpu_model_runner.py:2392] Model loading took 15.2683 GiB and 3.423974 seconds
(EngineCore_DP0 pid=571756) INFO 10-06 15:24:08 [backends.py:539] Using cache directory: /home/wjyeo/.cache/vllm/torch_compile_cache/00dcc66913/rank_0_0/backbone for vLLM's torch.compile
(EngineCore_DP0 pid=571756) INFO 10-06 15:24:08 [backends.py:550] Dynamo bytecode transform time: 5.31 s
(EngineCore_DP0 pid=571756) INFO 10-06 15:24:11 [backends.py:161] Directly load the compiled graph(s) for dynamic shape from the cache, took 2.655 s
(EngineCore_DP0 pid=571756) INFO 10-06 15:24:12 [monitor.py:34] torch.compile takes 5.31 s in total
(EngineCore_DP0 pid=571756) INFO 10-06 15:24:13 [gpu_worker.py:298] Available KV cache memory: 104.82 GiB
(EngineCore_DP0 pid=571756) INFO 10-06 15:24:13 [kv_cache_utils.py:864] GPU KV cache size: 763,232 tokens
(EngineCore_DP

Capturing CUDA graphs (mixed prefill-decode, PIECEWISE): 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 67/67 [00:02<00:00, 23.96it/s]


(EngineCore_DP0 pid=571756) INFO 10-06 15:24:16 [gpu_model_runner.py:3118] Graph capturing finished in 3 secs, took 0.77 GiB
(EngineCore_DP0 pid=571756) INFO 10-06 15:24:16 [gpu_worker.py:391] Free memory on device (139.21/139.81 GiB) on startup. Desired GPU memory utilization is (0.9, 125.83 GiB). Actual usage is 15.27 GiB for weight, 5.68 GiB for peak activation, 0.07 GiB for non-torch memory, and 0.77 GiB for CUDAGraph memory. Replace gpu_memory_utilization config with `--kv-cache-memory=111556550553` to fit into requested memory, or `--kv-cache-memory=125921838592` to fully utilize gpu memory. Current kv cache memory in use is 112544309145 bytes.
(EngineCore_DP0 pid=571756) INFO 10-06 15:24:16 [core.py:218] init engine (profile, create kv cache, warmup model) took 13.99 seconds
INFO 10-06 15:24:17 [llm.py:295] Supported_tasks: ['generate']
INFO 10-06 15:24:17 [__init__.py:36] No IOProcessor plugins requested by the model


ERROR 10-06 15:41:32 [core_client.py:564] Engine core proc EngineCore_DP0 died unexpectedly, shutting down client.


In [5]:
sampling_params = SamplingParams(temperature=0., max_tokens=2048, stop=tokenizer.eos_token)

# ASIDE/ISE

For approaches with learnt embeddings, we first get the SFT dataset, by just getting on-policy rollouts on the full alpaca

In [6]:
data_with_injection = []
for i,sample in enumerate(clean_data):
    if sample.get('input', '').strip() == '':
        continue
    data_with_injection.append(sample)
print (len(data_with_injection))

19157


In [ ]:
sft_inputs = []
for sample in data_with_injection:
    conv = [
        {"role": "user", "content": sample["instruction"] + "\n\n" + sample["input"]}
    ]
    sft_inputs.append(tokenizer.apply_chat_template(conv, tokenize=False, add_generation_prompt=True,enable_thinking=False))
print (f'SFT dataset size: {len(sft_inputs)}')  
sft_outputs = model.generate(sft_inputs, sampling_params,use_tqdm=True)
sft_outputs = [o.outputs[0].text for o in sft_outputs]

sft_data = []
for sample,output in zip(data_with_injection,sft_outputs):
    sft_data.append({
        'instruction': sample['instruction'],
        'input': sample['input'],
        'output': output
    })
    
val_size= 500
random_ids = np.random.permutation(len(sft_data))
sft_data_train = [sft_data[i] for i in random_ids[val_size:]]
sft_data_val = [sft_data[i] for i in random_ids[:val_size]]
print (f'SFT train size: {len(sft_data_train)}, val size: {len(sft_data_val)}')

sft_data_path = {'train':os.path.join(data_dir, "alpaca_rollout_train.json"),
                        'val':os.path.join(data_dir, "alpaca_rollout_val.json"),}

jdump(sft_data_train,sft_data_path['train'])
jdump(sft_data_val,sft_data_path['val'])

## Get length
input_len = [len(ids) for ids in tokenizer([d['instruction'] + '\n\n' + d['input'] for d in sft_data_train])["input_ids"]]
completion_len = [len(ids) for ids in tokenizer([d['response'] for d in sft_data_train])["input_ids"]]
overall_len = [i+c for i,c in zip(input_len,completion_len)]
print (f'Mean/Max/Min/>2048: {np.mean(overall_len):.2f} / {np.max(overall_len)} / {np.min(overall_len)} / {np.sum(np.array(overall_len)>2048)}')


SFT dataset size: 19157


Adding requests:   0%|          | 0/19157 [00:00<?, ?it/s]

Processed prompts:   0%|                                                                                      …

SFT train size: 18657, val size: 500
Mean/Max/Min/>2048: 248.08 / 2468 / 7 / 51


In [ ]:
preference_data = []
for i,sample in tqdm(enumerate(data_with_injection),total = len(data_with_injection)):
    current_sample = deepcopy(sample)
    injected_sample = deepcopy(sample)
    instruction = current_sample['instruction']
    inpt = current_sample['input']

    while injected_sample['instruction'] == current_sample['instruction']:
        injected_sample = np.random.choice(data_with_injection, size=1, replace=False)[0]
    
    injected_prompt = injected_sample['instruction'] + ' ' + injected_sample['input']
    if np.random.rand() < 0.9:  # 90% Straightforward Attack, 10% Completion Attack
        current_sample['input'] = injected_prompt + ' ' + current_sample['input'] if np.random.rand() < 0.5 else current_sample['input'] + ' ' + injected_prompt
    else: 
        fake_response = current_sample['output']
        current_sample['input'] += '\n\n' + create_injection_for_completion(fake_response, injected_sample['instruction'], injected_sample['input'])
    preference_data.append({
                'instruction': current_sample['instruction'],
                'input': current_sample['input'],
                'chosen_input': instruction + '\n\n' + inpt,
                'rejected_input': injected_sample['instruction'] + ' ' + injected_sample['input'],
            })
print (len(preference_data))

19157


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19157/19157 [00:24<00:00, 784.76it/s]

19157


In [ ]:
sampling_params = SamplingParams(temperature=0.8, max_tokens=2048, stop=tokenizer.eos_token)
conversations = []
for sample in preference_data:
    conversations.append(tokenizer.apply_chat_template([{"role": "user", "content": sample["chosen_input"]}], tokenize=False, add_generation_prompt=True,enable_thinking=False))
    conversations.append(tokenizer.apply_chat_template([{"role": "user", "content": sample["rejected_input"]}], tokenize=False, add_generation_prompt=True,enable_thinking=False))
outputs = model.generate(conversations, sampling_params,use_tqdm=True)
for i in range(len(preference_data)):
    sample = preference_data[i]
    sample['chosen'] = outputs[2*i].outputs[0].text + tokenizer.eos_token
    sample['rejected'] = outputs[2*i+1].outputs[0].text + tokenizer.eos_token

val_size = 500
preference_data_ids = np.random.permutation(range(len(preference_data)))
train_dataset = [preference_data[i] for i in range(len(preference_data)) if i not in preference_data_ids[:val_size]]
val_dataset = [preference_data[i] for i in preference_data_ids[:val_size]]

jdump(train_dataset, preference_data_path['train'])
jdump(val_dataset, preference_data_path['val'])
train_dataset = load_dataset('json', data_files=preference_data_path['train'], split='train')
val_dataset = load_dataset('json', data_files=preference_data_path['val'], split='train')
calculate_length_for_preference_dataset(train_dataset, tokenizer)
calculate_length_for_preference_dataset(val_dataset, tokenizer)